In [2]:
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

--2024-05-19 10:05:34--  https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/fenicsx-install-real.sh
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fem-on-colab/fem-on-colab.github.io/a312183439cf07dbe5561fe31ebee6ccd1f77324/releases/fenicsx-install-real.sh [following]
--2024-05-19 10:05:34--  https://raw.githubusercontent.com/fem-on-colab/fem-on-colab.github.io/a312183439cf07dbe5561fe31ebee6ccd1f77324/releases/fenicsx-install-real.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4319 (4.2K) [text/plain]
Saving to: ‘/tmp/fenicsx-install.sh’

/tmp/fenicsx-instal 100%[==

In [3]:
try:
    import viskex
except ImportError:
    !pip3 install "viskex@git+https://github.com/viskex/viskex.git@64c23fe"
    import viskex

  Cloning https://github.com/viskex/viskex.git (to revision 64c23fe) to /tmp/pip-install-aloy4tza/viskex_46a9f3b0ce0f4df88ef185ae0a089a7b
  Running command git clone --filter=blob:none --quiet https://github.com/viskex/viskex.git /tmp/pip-install-aloy4tza/viskex_46a9f3b0ce0f4df88ef185ae0a089a7b
  Running command git checkout -q 64c23fe
  Resolved https://github.com/viskex/viskex.git to commit 64c23fe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.6 MB/s eta 0:00:00
  Created wheel for viskex: filename=viskex-0.0.dev1-py3-none-any.whl size=22891 sha256=09025f681c0d051341ad88656dea0da47050dc71c19f7c930a85bb1bc288966c
  Stored in directory: /tmp/pip-ephem-wheel-cache-zqj30q44/wheels/28/9e/ea/48f255983e2a985971e7b543c2cdfdd51df4273800b0fc726c
Successfully built viskex


In [4]:
try:
    import gmsh
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

--2024-05-19 10:07:12--  https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/gmsh-install.sh
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fem-on-colab/fem-on-colab.github.io/a312183439cf07dbe5561fe31ebee6ccd1f77324/releases/gmsh-install.sh [following]
--2024-05-19 10:07:12--  https://raw.githubusercontent.com/fem-on-colab/fem-on-colab.github.io/a312183439cf07dbe5561fe31ebee6ccd1f77324/releases/gmsh-install.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3495 (3.4K) [text/plain]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   3.41K

In [5]:
import basix.ufl
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import numpy.typing
import petsc4py.PETSc
import ufl
from mpi4py import MPI


In [6]:
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 20, 20)

# Create connectivities required by the rest of the code
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)

In [7]:
# velocity space element
V_element = basix.ufl.element("Lagrange", mesh.basix_cell(), 1, shape=(mesh.geometry.dim, ))
# tensor space element
Q_element = basix.ufl.element("Lagrange", mesh.basix_cell(), 1, shape=(mesh.geometry.dim,mesh.geometry.dim), symmetry=True)

In [8]:
# define the function space
V = dolfinx.fem.functionspace(mesh, V_element)
Q = dolfinx.fem.functionspace(mesh, Q_element)

In [ ]:
#!zip -r /content/file.zip /content/test_u.bp
#from google.colab import files
#files.download("/content/file.zip")


  adding: content/test_u.bp/ (stored 0%)
  adding: content/test_u.bp/md.idx (deflated 59%)
  adding: content/test_u.bp/data.0 (deflated 82%)
  adding: content/test_u.bp/mmd.0 (deflated 64%)
  adding: content/test_u.bp/md.0 (deflated 74%)
  adding: content/test_u.bp/profiling.json (deflated 60%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# interpolate velocity in the nodal value
def u_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((2, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])
    values[1, :] = -np.pi*x[1]* np.cos(np.pi*x[1])
    return values

u = dolfinx.fem.Function(V,name="velocity")
u.interpolate(u_eval)

with dolfinx.io.VTXWriter(mesh.comm,  f"test_u.bp", u) as vtx_u:
    vtx_u.write(0)
#https://fenicsproject.discourse.group/t/numerical-values-from-ufl-spatialcoordinate/11064/2

In [10]:
def tau11_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])
    return values
def tau12_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = - np.pi*x[1]*np.cos(np.pi*x[0])
    return values
def tau22_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])*np.cos(np.pi*x[1])
    return values

def tau_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    values = np.zeros((4, x.shape[1]))
   # using python reshape A B, B C become A B B C
    values[0, :] = tau11_eval(x)
    values[1, :] = tau12_eval(x) # tau12_eval(x)
    values[2, :] = tau12_eval(x)
    values[3, :] = tau22_eval(x)

    return values



In [11]:
q_D = dolfinx.fem.Function(Q,name="stress")
q_D.interpolate(tau_eval)
fdim = mesh.topology.dim - 1
boundary_facets = dolfinx.mesh.locate_entities_boundary(mesh, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
bc_D = dolfinx.fem.dirichletbc(q_D, dolfinx.fem.locate_dofs_topological(Q, fdim, boundary_facets))



In [12]:
psi = ufl.TestFunction(Q)
tau = ufl.TrialFunction(Q)
dx=ufl.Measure("dx", domain=mesh, metadata={"quadrature_degree": 10})
Wi = 0.1
dt = 0.01

In [13]:
# Create initial condition
q_n = dolfinx.fem.Function(Q)
q_n.name = "q_n"
dolfinx.fem.set_bc(q_n.vector, [bc_D])

with dolfinx.io.XDMFFile(mesh.comm, "initialStress.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(q_n)


In [ ]:

formulation_one = False
if formulation_one:

  F = ufl.inner(tau,psi)*dx - dt*ufl.inner(ufl.dot(2*ufl.skew(ufl.grad(u)), tau),psi)*dx
  # transport part
  F+= dt*ufl.inner(ufl.dot(u,ufl.grad(tau)), psi)*dx

  # to complete here there is kappa factor..
  h = ufl.CellDiameter(mesh)
  beta = 1e-2
  stab  = beta* dt*h* ufl.inner(ufl.grad(tau),ufl.grad(psi))*dx
  F+=stab

  # see stabilization implemented PLS for the stress- need stabilization since it esplode!!
  # PhD thesis
  #https://orca.cardiff.ac.uk/id/eprint/118620/1/Theoretical%20and%20Computational%20Modelling%20of%20Compressible%20Viscoelastic%20Fluids%20-%20Alex%20Mackay%202018.pdf
  # Code
  #https://github.com/ATMackay/fenics/blob/master/lid-driven-cavity/comp_LDC.py
  L = 1/Wi *dt* (ufl.inner(ufl.Identity(mesh.geometry.dim),psi)*dx - ufl.inner(2*ufl.skew(ufl.grad(u)),psi)*dx  )
  L += 1/Wi* ufl.inner(q_n,psi)*dx

In [21]:
def ConvectStress(u, tau):
   return ufl.dot(u,ufl.grad(tau))- ufl.dot(ufl.grad(u), tau) - ufl.dot(tau, ufl.grad(u))

lhs = (Wi/dt+1)*tau + Wi* ConvectStress(u,tau)
rhs = Wi/dt* q_n + ufl.Identity(len(u))
F = ufl.inner(lhs, psi)*dx
L = ufl.inner(rhs,psi)*dx
# SUPG stabilization
h = ufl.CellDiameter(mesh)
magnitudeVel = ufl.sqrt(ufl.dot(u,u))

reg =1e-6
alpha_supg = h /(magnitudeVel+reg)
supg = ufl.inner( Wi*ufl.dot(u,ufl.grad(tau)), alpha_supg *ufl.dot(u, ufl.grad(psi)) )*dx
F += supg



In [22]:
from dolfinx.fem.petsc import LinearProblem

xdmf = dolfinx.io.XDMFFile(mesh.comm, "stressEvolution.xdmf", "w")
xdmf.write_mesh(mesh)

t = 0
vtx_u = dolfinx.io.VTXWriter(mesh.comm, "stress.bp", [q_n], engine="BP4")
vtx_u.write(t)

xdmf.write_function(q_n,t)
num_dt=10
for n in range(num_dt):
    t+=dt
    problem = LinearProblem(F, L, bcs=[bc_D], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    qh = problem.solve()
    qh.x.scatter_forward()

    # Update solution at previous time step (u_n)
    q_n.x.array[:] = qh.x.array
    q_n.name="stress"
    xdmf.write_function(q_n,t)
    vtx_u.write(t)
vtx_u.close()
xdmf.close()

In [25]:
# we need to do the static case now..

def convergence(uh,q_D):
  error_local = dolfinx.fem.assemble_scalar(dolfinx.fem.form((uh - q_D)**2 * ufl.dx))
  error_localH1 = dolfinx.fem.assemble_scalar(dolfinx.fem.form((uh -q_D)**2 * ufl.dx + ufl.grad(uh - q_D)**2 * ufl.dx) )


  error_L2 = numpy.sqrt(mesh.comm.allreduce(error_local, op=MPI.SUM))
  error_H1 =  numpy.sqrt(mesh.comm.allreduce(error_localH1, op=MPI.SUM))
  if mesh.comm.rank == 0:
      print(f"L2-error: {error_L2:.2e}")
      print(f"H1-error: {error_H1:.2e}")

  # Compute values at mesh vertices
  error_max = mesh.comm.allreduce(numpy.max(numpy.abs(uh.x.array - q_D.x.array)), op=MPI.MAX)
  if mesh.comm.rank == 0:
      print(f"Error_max: {error_max:.2e}")

#convergence(qh,q_D)

L2-error: 1.79e+00
H1-error: 2.07e+01
Error_max: 3.70e+00


In [1]:
pip list

Package                       Version
----------------------------- -----------
alabaster                     0.7.13
anyio                         4.0.0
argon2-cffi                   23.1.0
argon2-cffi-bindings          21.2.0
arrow                         1.3.0
asttokens                     2.4.1
async-lru                     2.0.4
attrs                         23.1.0
Babel                         2.13.0
beautifulsoup4                4.12.2
bleach                        6.1.0
certifi                       2023.7.22
cffi                          1.16.0
charset-normalizer            3.3.0
clang-format                  17.0.2
cmakelang                     0.6.13
comm                          0.2.0
contourpy                     1.1.1
cppimport                     22.8.2
cycler                        0.12.0
debugpy                       1.8.0
decorator                     5.1.1
defusedxml                    0.7.1
docutils                      0.18.1
exceptiongroup                1.1.3
exec